In [1]:
import os
import numpy as np
import pandas
import scipy
import pickle
import copy
import math

import pypcd
from pypcd import pypcd

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D             # For 3D plot
from matplotlib.patches import FancyArrowPatch      # For 3D arrow

from numpy import cos, sin
from ceva import Ceva
from IPython import display

import struct
import rosbag
import rospy

from sensor_msgs.msg import PointCloud2, PointField
from geometry_msgs.msg import TransformStamped
from tf2_msgs.msg import TFMessage

import myutil.myutil as util
from myparproc.ParProc import ParProc
from myparproc.ProgressCheck import ProgressCheck

cloudpkl = 'cloud_noisy.pkl'
cloudbag = 'cloud_noisy.bag'

onesigma = 0.02

# Lidar extrinsic
T_B_L = [np.array([[ 1,  0,  0,  0  ],
                   [ 0,  1,  0,  0  ],
                   [ 0,  0,  1,  0  ],
                   [ 0,  0,  0,  1  ]]),
         np.array([[-1,  0,  0, -0.5],
                   [ 0, -1,  0,  0  ],
                   [ 0,  0,  1, -0.5],
                   [ 0,  0,  0,  1  ]]),
        ]

# Number of lidars
Nl = len(T_B_L)

# The 6 planes
xb = 6
yb = 6
zb = 3
planes = [(np.array([1, 0, 0]).reshape((3, 1)), np.array([ xb, 0,  0 ]).reshape((3, 1))),
          (np.array([1, 0, 0]).reshape((3, 1)), np.array([-xb, 0,  0 ]).reshape((3, 1))),
          (np.array([0, 1, 0]).reshape((3, 1)), np.array([ 0,  yb, 0 ]).reshape((3, 1))),
          (np.array([0, 1, 0]).reshape((3, 1)), np.array([ 0, -yb, 0 ]).reshape((3, 1))),
          (np.array([0, 0, 1]).reshape((3, 1)), np.array([ 0,  0,  zb]).reshape((3, 1))),
          (np.array([0, 0, 1]).reshape((3, 1)), np.array([ 0,  0,  0 ]).reshape((3, 1)))]

# Generate the trajectory
dt = 1e-5
tmax = 69
tstart = 5

# Radius
r = 2
# Frequency
w = 0.25
# Time span
t = np.arange(0, tmax, dt)

# x y z coordinates of the body
p_W_B = []; R_W_B = []; x = []; y = []; z = []; vx = []; vy = []; vz = []
for time in t:
    if time < tstart:
        # Position
        x.append(0)
        y.append(0)
        z.append(1)
        # Velocity
        vx.append(0)
        vy.append(0)
        vz.append(0)
        # Orientation
        yaw = math.pi/4
        rot = np.array([[cos(yaw), -sin(yaw), 0],
                        [sin(yaw),  cos(yaw), 0],
                        [       0,         0, 1]])
        R_W_B.append(rot)
    else:
        tprime = time-tstart
        # Position
        x.append(2*sin(w*tprime))
        y.append(2*sin(w*tprime)*cos(w*tprime))
        z.append(1)
        # Velocity
        vx.append(2*w*cos(w*tprime))
        vy.append(2*w*(cos(w*tprime)*cos(w*tprime) - sin(w*tprime)*sin(w*tprime)))
        vz.append(0)
        # Orientation
        yaw = np.arctan2(vy[-1], vx[-1])
        rot = np.array([[cos(yaw), -sin(yaw), 0],
                        [sin(yaw),  cos(yaw), 0],
                        [       0,         0, 1]])
        R_W_B.append(rot)

p_W_B = [np.array([x_, y_, z_]).reshape(3, 1) for x_, y_, z_ in zip(x, y, z)]

# Beam direction
er = cos(57*t)
eu = er*cos(15.43*t)
ev = er*sin(15.43*t)


In [2]:
# Get lidar pose
def lidarPose(p, R, T):
    pl = p + R*T[:3, 3]
    Rl = R*T[:3, :3]
    return (pl, Rl)

# Return the point on the wall that a beam hits
def findHitpoint(p, e, std_dev=0):
    # Check if the ray hits any of the planes
    for n, m in planes:
        nm_ = np.squeeze(n.T@m)
        np_ = np.squeeze(n.T@p)
        ne_ = np.squeeze(n.T@e)
        eta = np.random.normal(0, std_dev, 1)
        if ne_ == 0:
            continue
        else:
            s = (nm_ - np_)/ne_
            h = p + (s + eta)*e
            if s > 0 and -xb <= h[0] <= xb and -yb <= h[1] <= yb and 0 <= h[2] <= zb:
                return s, h, ne_.item()
    return 0, p, 0

In [3]:
# # Plot the trajectory

# figpos = [18, 6]
# fig, ax = plt.subplots(nrows=1, ncols=3, figsize=figpos)

# # ax[0].clear()
# # ax[0] = fig.add_subplot(111, projection='3d')
# ax[0].plot(x, y)
# ax[0].set_xlabel('x [m]')
# ax[0].set_ylabel('y [m]')
# # ax[0].set_zlabel('z [m]')

# ax[0].grid('on')

# # plot the time
# vxline, vyline = None, None
# ax[1].set_xlabel('vx [m/s]')
# ax[1].set_ylabel('vy [m/s]')
# ax[1].grid('on')
# ax[1].set_ylim([np.min(vx)*1.2, np.max(vy)*1.2])

# # Plot the flower
# flowerline1, = ax[2].plot(eu[0], ev[0], '2r')
# flowerline2, = ax[2].plot(eu[0], ev[0], 'b')

# ax[2].set_aspect('equal')
# ax[2].set_xlim([-1.2, 1.2])
# ax[2].set_ylim([-1.2, 1.2])

# # Plot the velcity at t = 69
# arrow = None
# idx_prev = 0
# idx_curr = 0
# for idx_curr, time in enumerate(t):
    
#     # idx = np.where(t==time)[0][0]
#     if (idx_curr - idx_prev)*dt < 0.1:
#         continue

#     vx_ = vx[idx_curr]
#     vy_ = vy[idx_curr]
#     px_ =  x[idx_curr]
#     py_ =  y[idx_curr]

#     if arrow is not None:
#         arrow.remove()
#     arrow = ax[0].arrow(px_, py_, vx_, vy_, width=0.01, head_width=0.05, head_length=0.1, fc='blue', ec='blue')

#     if vxline is not None or vyline is not None:
#         vxline.remove()
#         vyline.remove()
#     vxline, = ax[1].plot(t[:idx_curr], vx[:idx_curr], 'r')
#     vyline, = ax[1].plot(t[:idx_curr], vy[:idx_curr], 'b')

#     flowerline1.set_data(eu[idx_prev:idx_curr], ev[idx_prev:idx_curr])
#     flowerline2.set_data(eu[0:idx_curr], ev[0:idx_curr])

#     idx_prev = idx_curr
    
#     display.clear_output(wait=True)
#     display.display(plt.gcf())
    

In [4]:
# # Generate the point for each lidar over the trajectory

# clouds = []
# for n in range(Nl):
#     clouds.append([])

# def makeCloudData(lidx, proc_idx=None, output_queue=None):
    
#     # Create the progress checker
#     pck = ProgressCheck(len(t))
    
#     # List to store the points
#     cloud = []
    
#     # Get the extrinsics
#     R_B_L, t_B_L = T_B_L[lidx][:3, :3], T_B_L[lidx][:3, 3].reshape(3, 1)

#     for idx, stamp in enumerate(t):

#         # Find the lidar rotation
#         R_W_L = R_W_B[idx] @ R_B_L
#         q_W_L = util.rotm2quat(R_W_L)

#         # Find the lidar position
#         p_W_L = p_W_B[idx] + R_W_B[idx] @ t_B_L

#         # Find beam direction
#         e_inL = np.array([1, eu[idx], ev[idx]]).reshape(3, 1)
#         e_inL = e_inL / np.linalg.norm(e_inL)
#         e_inW = R_W_L@e_inL

#         # Find the hit point
#         s, h_inW, cosne = findHitpoint(p_W_L, e_inW, onesigma)

#         # Transform the point to lidar frame
#         h_inL = R_W_L.T@(h_inW - p_W_L)

#         # Extract the elements
#         # px, py, pz = h_inL[0].item(), h_inL[1].item(), h_inL[2].item()
#         # qx, qy, qz, qw = q_W_L[1].item(), q_W_L[2].item(), q_W_L[3].item(), q_W_L[0].item()

#         # Store the point
#         cloud.append([stamp, h_inL, p_W_L, R_W_L, cosne])

#         # Update the progress
#         pck.updateProgress(f'Lidar {lidx} Generation: ')

#     if proc_idx is not None and output_queue is not None and len(output_queue) > proc_idx:
#         output_queue[proc_idx] = cloud

# # Add the jobs
# pp = ParProc(Nl)
# for lidx in range(Nl):
#     pp.addprocwithoutput(makeCloudData, lidx, lidx)
#     # makeCloudData(lidx, lidx, clouds)
# pp.join()

# # Extract the output
# clouds = list(pp.output_queue)

# # Dump the data
# with open(cloudpkl, 'wb') as f:
#     pickle.dump(clouds, f)


In [5]:
# Load the data
with open(cloudpkl, 'rb') as file:
    clouds = pickle.load(file)


In [6]:
# Export the data to pointcloud

# Create the rosbag
bag = rosbag.Bag(cloudbag, 'w')

pck = ProgressCheck(len(t))
# Loop through the points and write the pointcloud
for lidx, points in enumerate(clouds):
    Npoints = len(points)
    for bidx in range(0, Npoints, int(1e4)):
        
        scan = []
        t0 = clouds[lidx][bidx][0]

        for idx in range(bidx, bidx + int(1e4)):
            pt = int((clouds[lidx][idx][0] - t0)*1e9)
            px = clouds[lidx][idx][1][0]
            py = clouds[lidx][idx][1][1]
            pz = clouds[lidx][idx][1][2]
            pi = abs(1.0 - clouds[lidx][idx][-1])*1000
            scan.append((px, py, pz, pi, pt))

        p_W_L = clouds[lidx][bidx][2]
        R_W_L = clouds[lidx][bidx][3]
        q_W_L = util.rotm2quat(R_W_L)
        
        # Convert scan to pointcloud msg
        scan = np.array(scan, dtype=[('x', '<f4'), ('y', '<f4'), ('z', '<f4'), ('intensity', '<f4'), ('t', '<u4')])
        scan_msg = pypcd.PointCloud.from_array(scan).to_msg()
        scan_msg.header.stamp = rospy.Time(t0)
        scan_msg.header.frame_id = f'lidar_{lidx}'

        # Create a tf message
        transform_msg_config = TFMessage()
        transform_msg = TransformStamped()
        transform_msg.header.stamp = rospy.Time(t0)
        transform_msg.header.frame_id = 'world'
        transform_msg.header.seq = int(bidx % 1e4)
        transform_msg.child_frame_id = f'lidar_{lidx}'
        transform_msg.transform.translation.x = p_W_L[0]
        transform_msg.transform.translation.y = p_W_L[1]
        transform_msg.transform.translation.z = p_W_L[2]
        transform_msg.transform.rotation.x = q_W_L[1]
        transform_msg.transform.rotation.y = q_W_L[2]
        transform_msg.transform.rotation.z = q_W_L[3]
        transform_msg.transform.rotation.w = q_W_L[0]
        transform_msg_config.transforms.append(transform_msg)
        
        bag.write(f'/lidar_{lidx}/points', scan_msg, rospy.Time(t0))
        bag.write(f'/tf', transform_msg_config, rospy.Time(t0))

# Close the bag
bag.close()


In [7]:
!rosbag play $cloudbag & rviz -d /home/tmn/ros_ws/dev_ws/src/cartiber/scripts/cartiber.rviz


[ INFO]: rviz version 1.14.20
[ INFO]: compiled against Qt version 5.12.8
[ INFO]: compiled against OGRE version 1.9.0 (Ghadamon)
[ INFO]: Forcing OpenGl version 0.
[ INFO]: Opening cloud_noisy.bag
Error:   Index entry for topic /lidar_0/points contains invalid time.  This message will not be loaded.
         at line 651 in /tmp/binarydeb/ros-noetic-rosbag-storage-1.16.0/src/bag.cpp
Error:   Index entry for topic /tf contains invalid time.  This message will not be loaded.
         at line 651 in /tmp/binarydeb/ros-noetic-rosbag-storage-1.16.0/src/bag.cpp
Error:   Index entry for topic /tf contains invalid time.  This message will not be loaded.
         at line 651 in /tmp/binarydeb/ros-noetic-rosbag-storage-1.16.0/src/bag.cpp
Error:   Index entry for topic /lidar_1/points contains invalid time.  This message will not be loaded.
         at line 651 in /tmp/binarydeb/ros-noetic-rosbag-storage-1.16.0/src/bag.cpp

Waiting 0.2 seconds after advertising topics... done.

Hit space to toggl

In [ ]:
# import math

# # Generate the priormap

# da = 0.5

# psi_ = list(np.arange(0, 360, da))
# theta_ = list(np.arange(-60, 60, da))

# p0 = np.array([0, 0, 1.5]).reshape((3, 1))
# priormap = []

# pck = ProgressCheck(len(psi_))
# for yaw_deg in psi_:
#     for pitch_deg in theta_:
        
#         # Angles in radiance
#         yaw_rad = yaw_deg*math.pi/180
#         pitch_rad = pitch_deg*math.pi/180

#         # Beam bearing
#         ex = cos(yaw_rad)*cos(pitch_rad)
#         ey = sin(yaw_rad)*cos(pitch_rad)
#         ez = sin(pitch_rad)
#         e  = np.array([ex, ey, ez]).reshape((3, 1))

#         # Find the hit point
#         s, h, cosne = findHitpoint(p0, e, 0.001)

#         # Extract the x y z intensity
#         x = h[0].item()
#         y = h[1].item()
#         z = h[2].item()
#         intensity = abs(cosne)*1000

#         # Save the point
#         priormap.append((x, y, z, intensity))
    
#     # Update the progress
#     pck.updateProgress('Generating priormap: ')

# # Convert to array
# priormap = np.array(priormap, dtype=[('x', '<f4'), ('y', '<f4'), ('z', '<f4'), ('intensity', '<f4')])

# # Save the pointcloud
# pypcd.save_point_cloud_bin(pypcd.PointCloud.from_array(priormap), 'priormap.pcd')

In [11]:
clouds[1][0]

[0.0,
 array([[3.97614678],
        [3.97614678],
        [0.        ]]),
 array([[-0.35355339],
        [-0.35355339],
        [ 0.5       ]]),
 array([[-0.70710678,  0.70710678,  0.        ],
        [-0.70710678, -0.70710678,  0.        ],
        [ 0.        ,  0.        ,  1.        ]]),
 -0.9999999999999999]